In [1]:
!pip install PyMySQL

In [1]:
# from flask import Flask,render_template,request  ## flask 라이브러리에서 Flask import
# import pymysql
# import pandas as pd

In [24]:
# app = Flask(__name__)

# con = pymysql.connect(host='project-db-campus.smhrd.com', user='sc_24K_bigdata25_p3_4', password='smhrd4',
#                       port=3312, db='sc_24K_bigdata25_p3_4', charset='utf8', # 한글처리 (charset = 'utf8')
#                       autocommit=True # 결과 DB 반영 (Insert or update)
#                      )
# cur = con.cursor()

# # @app.route('/boot/ex/<i nt:seqidx>/<user_id>')
# # def hello_world(seqidx,user_id):
# #     return render_template('eltag.html')

# #@app.route('/boot/ex/<user_id>')
# #def hello_world():
# #    return render_template('hello2.html')
# @app.route('/boot/ex/<id>')
# def test_get(id):
#     print(id);
#     return render_template('index10.html', id=id)


# if __name__ == "__main__":
#     app.run(host='localhost', port=8097)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8097
Press CTRL+C to quit
127.0.0.1 - - [09/Dec/2024 11:46:22] "GET /boot/ex HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2024 11:46:25] "GET /boot/ HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2024 11:46:28] "GET /boot HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2024 11:46:39] "GET /boot/ex/vncj HTTP/1.1" 200 -


vncj


127.0.0.1 - - [09/Dec/2024 11:46:39] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Dec/2024 11:46:39] "GET /video_feed HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2024 11:46:39] "GET /boot/img/아동자세.png HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2024 11:50:55] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Dec/2024 11:51:50] "GET /boot/ex/vncj HTTP/1.1" 200 -


vncj


127.0.0.1 - - [09/Dec/2024 11:51:51] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Dec/2024 11:51:51] "GET /video_feed HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2024 11:51:51] "GET /boot/img/아동자세.png HTTP/1.1" 404 -


In [3]:
os.getcwd()

'C:\\Users\\USER\\yogafire'

In [77]:
from flask import Flask, render_template, Response
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import datetime
import time
import pymysql
import os

In [78]:
#점수 조정을 위한 함수 (수정가능)

def sigmoid(x):
    return 1 / (1 + np.exp(-x+1))

In [83]:
from flask import Flask, render_template, Response
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import datetime
import time
import pymysql
import os

#점수 조정을 위한 함수 (수정가능)

def sigmoid(x):
    return 1 / (1 + np.exp(-x+1))

app = Flask(__name__)

con = pymysql.connect(host='project-db-campus.smhrd.com', user='sc_24K_bigdata25_p3_4', password='smhrd4',
                      port=3312, db='sc_24K_bigdata25_p3_4', charset='utf8', # 한글처리 (charset = 'utf8')
                      autocommit=True # 결과 DB 반영 (Insert or update)
                     )
cur = con.cursor()

# 랜드마크 선택
selected_landmarks = [0] + list(range(11, 17)) + list(range(23, 29))
    
# 컬럼 이름 정의
columns = ['frame'] + [f'landmark{idx}_{axis}' for idx in selected_landmarks for axis in ['x', 'y']] + \
              ['neck_x', 'neck_y', 'hip_x', 'hip_y']


def generate_frames(id, ps_idx, flow_order_idx):

    # MediaPipe 초기화
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    print('generate_frames 실행' ,id , ps_idx, flow_order_idx)
    user_id = id
    select_yoga = ps_idx
    data = []
    
    try:
        cap = cv2.VideoCapture(0)  # 카메라 열기
        frame_idx = 0
        start_time = time.time()
        duration = 10  # 30초 동안 실행
    
        while cap.isOpened() and time.time() - start_time < duration:
            ret, frame = cap.read()
            if not ret:
                break
    
            frame_idx += 1
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(rgb_frame)
            frame_data = [frame_idx]
    
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                for idx in selected_landmarks:
                    landmark = landmarks[idx]
                    frame_data.extend([landmark.x, landmark.y])
    
                # 목과 엉덩이 좌표 계산
                neck_x = (landmarks[11].x + landmarks[12].x) / 2
                neck_y = (landmarks[11].y + landmarks[12].y) / 2
                hip_x = (landmarks[23].x + landmarks[24].x) / 2
                hip_y = (landmarks[23].y + landmarks[24].y) / 2
                frame_data.extend([neck_x, neck_y, hip_x, hip_y])
    
            else:
                frame_data.extend([None, None] * len(selected_landmarks))
                frame_data.extend([None, None, None, None])
    
            data.append(frame_data)
    
            # 시각화
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                h, w, _ = frame.shape
    
                # 점 그리기
                for idx in selected_landmarks:
                    x, y = int(landmarks[idx].x * w), int(landmarks[idx].y * h)
                    cv2.circle(frame, (x, y), 5, (255, 0, 0), -1)  # 파란색 점
    
                # 선 그리기
                for connection in mp_pose.POSE_CONNECTIONS:
                    start_idx, end_idx = connection
                    if start_idx in selected_landmarks and end_idx in selected_landmarks:
                        start_x, start_y = int(landmarks[start_idx].x * w), int(landmarks[start_idx].y * h)
                        end_x, end_y = int(landmarks[end_idx].x * w), int(landmarks[end_idx].y * h)
                        cv2.line(frame, (start_x, start_y), (end_x, end_y), (255, 255, 255), 2)  # 하얀색 선
    
            _, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
    
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

    except Exception as e:
        print(f"오류 발생: {e}")
    finally:
        cap.release()  # 자원 해제
        pose.close()  # MediaPipe 종료
    
    # CSV 저장
    suffix = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    output_csv_path = f'./BPE/training/{user_id}_{suffix}.csv'
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output_csv_path, index=False)
    print(f"Joint 데이터를 {output_csv_path}에 저장했습니다.")

    #BPE 안의 모델 파일 실행 / training 안에 방금 저장된 유저의 파일과 , rightyogajoint에 미리 저장해둔 요가 자세의 올바른 자세 joint 값과 비교 
    output = os.popen(f"python BPE/inference_pair.py -v1 BPE/training/{user_id}_{suffix}.csv -v2 BPE/rightyogajoint/{select_yoga}.csv").read()
    #실행된 결과에서 score만 가져오기
    list1 = output.split('\n')
    score_str = list1[-2]
    # score 점수 조정
    model_score = ((round(sigmoid((float(score_str.split(' ')[-1])))*10)*10))
    activity_at = datetime.datetime.now().strftime('%Y-%m-%d')  # 활동 시간
    activity_score = model_score  # 예시 점수
    insert_user_data(id, flow_order_idx, activity_at, activity_score)
    


def recent_order(id, flow_idx) :
    # activity_at -> datetime으로 바꿀수잇는지 물어보기
    sql = 'select flow_ord from tmp where (id = %s) and (flow_idx = %s) limit 1'
    cur.execute(sql, (id,flow_idx))
    flow_ord = cur.fetchone()
    # sql2 = 'select max(flow_ord) from flow_order where flow_idx = %s group by flow_idx;'
    # cur.excute(sql2, (flow_idx,))
    # last_ord = cur.fetchone()
    if flow_ord :
        # if flow_ord[0] == last_ord[0] :
        #     print('시퀀스를 마지막까지 실행한 유저입니다.')
        #     return 1
        # else : 
        print('가장 최근의 flow_ord :', flow_ord[0])
        return flow_ord[0] + 1
    else :
        print('id에 대한 최근 flow_ord가 없습니다.')
        return 1

def select_ps_data(flow_idx, flow_ord) :
    # flow_ord와 flow_idx에 해당하는 ps_idx를 찾는 쿼리
    sql = 'select ps_idx from flow_order where flow_idx = %s and flow_ord = %s'
    cur.execute(sql, (flow_idx, flow_ord))
    ps_idx = cur.fetchone()  # 쿼리 결과에서 첫 번째 값만 가져옵니다.  
    print(ps_idx)
    return ps_idx

def select_order_data(flow_idx, flow_ord) :
    print('select order data')
    sql = 'select flow_order_idx from flow_order where flow_idx = %s and flow_ord = %s'
    cur.execute(sql, (flow_idx, flow_ord))
    flow_order_idx = cur.fetchone()  # 쿼리 결과에서 첫 번째 값만 가져옵니다.
    print(flow_order_idx)
    return flow_order_idx

def select_flow_data(flow_idx) :
    sql = 'select * from flow_info where flow_idx = %s'
    cur.execute(sql, (flow_idx,))
    flow_info = cur.fetchall()
    print('select flow data 실행')
    return flow_info

def select_pose_data(ps_idx) :
    sql = 'select* from pose_info where ps_idx = %s'
    cur.execute(sql, (ps_idx,))
    ps_info = cur.fetchall()
    print('select pose data 실행')
    return ps_info

def insert_user_data(id, flow_order_idx, activity_at, activity_score) :
    print('insert user data')
    sql = 'insert into user_activity (id, flow_order_idx, activity_at, activity_score) values (%s, %s, %s, %s)'
    cur.execute(sql, (id, flow_order_idx, activity_at, activity_score))
    con.commit()


In [71]:
score_str = '0.7119442'
model_score = ((round(sigmoid((float(score_str.split(' ')[-1])))*10)*10))
print(model_score)

40


In [ ]:

@app.route('/video_feed/<id>/<ps_idx>/<flow_ord_index>')
def video_feed(id,ps_idx,flow_ord_index):
    return Response(generate_frames(id,ps_idx,flow_ord_index), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/boot/programs/<id>/<flow_idx>')
def index(id, flow_idx):

    print('flow_idx:',flow_idx)
    flow_ord = recent_order(id,flow_idx) # flow_idx, id로 들어온 플로우의 몇 번까지 실행했는지
    print('flow_ord:',flow_ord)
    ps_idx = select_ps_data(flow_idx, flow_ord)
    print(ps_idx)
    flow_order_idx = select_order_data(flow_idx, flow_ord)
    print('flow_order_idx:',flow_order_idx)

    flow_info = select_flow_data(flow_idx)
    pose_info = select_pose_data(ps_idx)

    return render_template('programs.html', id=id, flow_idx=flow_idx, flow_title = flow_info[0][2], 
                           ps_idx = pose_info[0][0],
                           ps_name = pose_info[0][1],  # '웃타나사나'
                           ps_time = pose_info[0][2],  # 30
                           ps_exp = pose_info[0][3],   # 자세 설명`
                           ps_img = pose_info[0][4],   # '1-1.jpg'
                           ps_cal = pose_info[0][5],   # 100
                           ps_link = pose_info[0][6],
                          flow_order_idx = flow_order_idx[0])  # 링크)
      
if __name__ == "__main__":
    app.run(host='localhost', port=8097)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8097
Press CTRL+C to quit
127.0.0.1 - - [12/Dec/2024 09:50:05] "GET /boot/programs/123456/1 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2024 09:50:05] "GET /static/css/programs.css HTTP/1.1" 200 -


flow_idx: 1
id에 대한 최근 flow_ord가 없습니다.
flow_ord: 1
(51,)
(51,)
select order data
(25,)
flow_order_idx: (25,)
select flow data 실행
select pose data 실행


127.0.0.1 - - [12/Dec/2024 09:50:06] "GET /static/css/fire.css HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2024 09:50:06] "GET /images/yogapose/10-8.jpg HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 09:50:06] "GET /static/js/programs.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2024 09:50:06] "GET /images/yogapose/10-6.jpg HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 09:50:06] "GET /images/yogapose/10-7.jpg HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 09:50:06] "GET /images/yogapose/10-4.jpg HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 09:50:06] "GET /images/yogapose/10-5-1.png HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 09:50:06] "GET /images/yogapose/1-1-1.png HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 09:50:06] "

generate_frames 실행 123456 51 25


...

[2024-12-12 09:50:49,438] ERROR in app: Exception on /boot/programs/123456/None [GET]
Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\envs\opencv\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\USER\anaconda3\envs\opencv\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\USER\anaconda3\envs\opencv\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\USER\anaconda3\envs\opencv\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_864\837964409.py", line 19, in index
    return render_template('programs.html', id=id, flow_idx=flow_idx, flow_title = flow_info[0][2],
IndexError: tuple index out of range

flow_idx: None
id에 대한 최근 flow_ord가 없습니다.
flow_ord: 1
None
None
select order data
None
flow_order_idx: None
select flow data 실행
select pose data 실행


GET /static/css/programs.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2024 09:50:49] "GET /static/css/fire.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2024 09:50:49] "GET /images/yogapose/10-3.jpg HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 09:50:49] "GET /static/js/programs.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2024 09:50:49] "GET /images/yogapose/10-7.jpg HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 09:50:49] "GET /images/yogapose/10-8.jpg HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 09:50:49] "GET /images/yogapose/10-6.jpg HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 09:50:49] "GET /images/yogapose/10-4.jpg HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 09:50:49] "GET /images/yogapose/10-5-1.png HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 09:50:50] "GET /images/yogapose/1-1-1.png HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 09:50:50] "GET /images/yogapose/10-2.jpg HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 09:50:55] "GET /video_feed/123456/50/26 HTTP/1.1" 200 -


Joint 데이터를 ./BPE/training/123456_20241212_095105.csv에 저장했습니다.


Error on request:
Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\envs\opencv\lib\site-packages\werkzeug\serving.py", line 370, in run_wsgi
    execute(self.server.app)
  File "C:\Users\USER\anaconda3\envs\opencv\lib\site-packages\werkzeug\serving.py", line 333, in execute
    for data in application_iter:
  File "C:\Users\USER\anaconda3\envs\opencv\lib\site-packages\werkzeug\wsgi.py", line 256, in __next__
    return self._next()
  File "C:\Users\USER\anaconda3\envs\opencv\lib\site-packages\werkzeug\wrappers\response.py", line 32, in _iter_encoded
    for item in iterable:
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_864\3161109226.py", line 121, in generate_frames
    model_score = ((round(sigmoid((float(score_str.split(' ')[-1])))*10)*10))
ValueError: could not convert string to float: 'ready'
